In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>'

SyntaxError: ignored

In [2]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')
# FindSpark simplifies the process of using Apache Spark with Python
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 18.0 MB/s eta 0:00:00


In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [4]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true
!mv shake.parquet?raw=true shake.parquet

--2023-09-05 07:43:51--  https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/claimed-framework/component-library/blob/master/coursera_ml/shake.parquet?raw=true [following]
--2023-09-05 07:43:51--  https://github.com/claimed-framework/component-library/blob/master/coursera_ml/shake.parquet?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/claimed-framework/component-library/raw/master/coursera_ml/shake.parquet [following]
--2023-09-05 07:43:51--  https://github.com/claimed-framework/component-library/raw/master/coursera_ml/shake.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubuserconten

In [5]:
df=spark.read.parquet('shake.parquet')

In [6]:
df.show()

+-----+---------+-----+-----+-----+
|CLASS| SENSORID|    X|    Y|    Z|
+-----+---------+-----+-----+-----+
|    2| qqqqqqqq| 0.12| 0.12| 0.12|
|    2|aUniqueID| 0.03| 0.03| 0.03|
|    2| qqqqqqqq|-3.84|-3.84|-3.84|
|    2| 12345678| -0.1| -0.1| -0.1|
|    2| 12345678|-0.15|-0.15|-0.15|
|    2| 12345678| 0.47| 0.47| 0.47|
|    2| 12345678|-0.06|-0.06|-0.06|
|    2| 12345678|-0.09|-0.09|-0.09|
|    2| 12345678| 0.21| 0.21| 0.21|
|    2| 12345678|-0.08|-0.08|-0.08|
|    2| 12345678| 0.44| 0.44| 0.44|
|    2|    gholi| 0.76| 0.76| 0.76|
|    2|    gholi| 1.62| 1.62| 1.62|
|    2|    gholi| 5.81| 5.81| 5.81|
|    2| bcbcbcbc| 0.58| 0.58| 0.58|
|    2| bcbcbcbc|-8.24|-8.24|-8.24|
|    2| bcbcbcbc|-0.45|-0.45|-0.45|
|    2| bcbcbcbc| 1.03| 1.03| 1.03|
|    2|aUniqueID|-0.05|-0.05|-0.05|
|    2| qqqqqqqq|-0.44|-0.44|-0.44|
+-----+---------+-----+-----+-----+
only showing top 20 rows



In [7]:
df.createOrReplaceTempView("df")

In [9]:
!pip install systemds==2.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 12.0 MB/s eta 0:00:00


In [10]:
from pyspark.sql.functions import monotonically_increasing_id
from systemds.context import SystemDSContext
import numpy as np
import pandas as pd

def dft_systemds(signal,name):


    with SystemDSContext(spark) as sds:
        size = signal.count()
        signal = sds.from_numpy(signal.toPandas().to_numpy())
        pi = sds.scalar(3.141592654)

        n = sds.seq(0,size-1)
        k = sds.seq(0,size-1)

        M = (n @ (k.t())) * (2*pi/size)

        Xa = M.cos() @ signal
        Xb = M.sin() @ signal

        index = (list(map(lambda x: [x],np.array(range(0, size, 1)))))
        DFT = np.hstack((index,Xa.cbind(Xb).compute()))
        DFT_pdf = pd.DataFrame(DFT, columns=list(["id",name+'_sin',name+'_cos']))
        DFT_df = spark.createDataFrame(DFT_pdf)
        return DFT_df


In [11]:
x0 = spark.sql("select X from df where Class=1") #=> Please create a DataFrame containing only measurements of class 0 from the x axis
y0 = spark.sql("select Y from df where Class=1") #=> Please create a DataFrame containing only measurements of class 0 from the y axis
z0 = spark.sql("select Z from df where Class=1") #=> Please create a DataFrame containing only measurements of class 0 from the z axis
x1 = spark.sql("select X from df where Class=2") #=> Please create a DataFrame containing only measurements of class 1 from the x axis
y1 = spark.sql("select Y from df where Class=2") #=> Please create a DataFrame containing only measurements of class 1 from the y axis
z1 = spark.sql("select Z from df where Class=2") #=> Please create a DataFrame containing only measurements of class 1 from th

In [12]:
from pyspark.sql.functions import lit

df_class_0 = dft_systemds(x0,'x') \
    .join(dft_systemds(y0,'y'), on=['id'], how='inner') \
    .join(dft_systemds(z0,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(0))

df_class_1 = dft_systemds(x1,'x') \
    .join(dft_systemds(y1,'y'), on=['id'], how='inner') \
    .join(dft_systemds(z1,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(1))

df_dft = df_class_0.union(df_class_1)

df_dft.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|   id|               x_sin|               x_cos|               y_sin|               y_cos|               z_sin|               z_cos|class|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|  8.0|0.007264897147586408|0.050779226648936575|0.007264897147586408|0.050779226648936575|0.007264897147586408|0.050779226648936575|    0|
| 67.0|-0.04728754200772285|-0.07005979881711377|-0.04728754200772285|-0.07005979881711377|-0.04728754200772285|-0.07005979881711377|    0|
| 70.0|-0.07768948937581313|-0.03913346753594...|-0.07768948937581313|-0.03913346753594...|-0.07768948937581313|-0.03913346753594...|    0|
|  0.0|               -0.13|                 0.0|               -0.13|                 0.0|               -0.13|                 0.0|    0|
| 69.0|-9.9001177621

In [13]:
from pyspark.ml.feature import VectorAssembler

In [19]:
vectorAssembler = VectorAssembler(inputCols=["x_sin", "x_cos", "y_sin", "y_cos", "z_sin", "z_cos"], outputCol="features")

In [20]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(labelCol="class", featuresCol="features", numTrees=10)

In [21]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [22]:
model = pipeline.fit(df_dft)
prediction = model.transform(df_dft)
prediction.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|   id|               x_sin|               x_cos|               y_sin|               y_cos|               z_sin|               z_cos|class|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|  8.0|0.007264897147586408|0.050779226648936575|0.007264897147586408|0.050779226648936575|0.007264897147586408|0.050779226648936575|    0|[0.00726489714758...|[8.68824100693077...|[0.86882410069307...|       0.0|
| 67.0|-0.04728754200772285|-0.07005979881711377|-0.04728754200772285|-0.07005979881711377|-0.04728754200772285|-0.07005979881711377|    0|[-0.0

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("class")

binEval.evaluate(prediction)

0.9981761070017225

In [24]:
!rm -Rf a2_m4.json

In [25]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m4.json')

In [26]:
!rm -f rklib.py
!wget wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2023-09-05 08:05:47--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2023-09-05 08:05:47--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2023-09-05 08:05:50 (28.7 MB/s) - ‘rklib.py’ saved [2540/2540]

FINISHED --2023-09-05 08:05:50--
Total wall clock time: 2.3s
Downloaded: 1 files, 2.5K in 0s (28.7 MB/s)


In [27]:
from rklib import zipit
zipit('a2_m4.json.zip','a2_m4.json')

In [28]:
!base64 a2_m4.json.zip > a2_m4.json.zip.base64

In [29]:
from rklib import submit
key = "-fBiYHYDEeiR4QqiFhAvkA"
part = "IjtJk"
email = "RavikantHatwar@gmail.com"
submission_token = "zcYlPIZLfz7VbH0n"

with open('a2_m4.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, submission_token, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"B8wXV","id":"f_F-qCtuEei_fRLwaVDk3g~B8wXV~ZRAKkkvDEe6skQ61mQIkMQ","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
